In [8]:
import pandas as pd
import numpy as np
from ipywidgets import *
from IPython.display import display, clear_output
import datetime
import os
import tkinter as tk
from tkinter import filedialog
import matplotlib
matplotlib.use('Agg')


pd.options.display.float_format = '{:,.2f}'.format
now = datetime.datetime.now()
end_date = str(now.month) + "/" + str(now.year)[-2:] if now.month >=10 else '0' + str(now.month) + "/" + str(now.year)[-2:]
beg_dict = {'01': ['01', '02', '03'], '04': ['04', '05', '06'], '07': ['07', '08', '09'], '10': ['10', '11', '12']}

beg_date_Q = [k for (k, v) in beg_dict.items() for i in range(0, len(v)) if end_date[:2] == v[i]][0]+'/'+ str(now.year)[-2:]
beg_date_Y = '01/'+str(now.year)[-2:]
date_dict = {'QTD': beg_date_Q, 'YTD': beg_date_Y}

data = pd.read_csv('test.csv', skiprows=2, names=['Account', 'Acc_Date', 'Acc_B', 'Acc_Raw', 
                                                                            'rom', 'len', 'num', 'text_date', 'date1',
                                                                           'date2', 'temp', 'temp2', 'temp3', 'temp4',
                                                                           'Dept', 'Item', 'Amt', 'Amt2', 'Gross',
                                                                           'temp5', 'temp6'])
columns = ['rom', 'len', 'num', 'date1', 'date2', 'temp', 'temp2', 'temp3', 'temp4', 'temp5', 'temp6']
data = data.drop(columns=columns)
data.dropna(inplace=True)
dept_dict = {'A': 'Acct', 'B':'Finance', 'C': 'Tax', 'D': 'Asset Mgmt', 'E': 'Port Mgmt', 'F': 'Acq', 'G': 'HR',
            'H': 'IT', 'I': 'Legal', 'J': 'Admin', 'K': 'Mkt', 'L': 'IR', 'M': 'IS', 'N': 'BDREX', 'O': 'Corp Mgmt', 
             'P': 'No Dept'}

data['Dep_Name'] = data['Dept'].apply(lambda x: dept_dict[x])

item = pd.read_csv('item.csv', names=['Account', 'Item_name'])

df = pd.merge(data, item, how='left', left_on='Acc_Raw', right_on='Account')
df['Total'] = df['Amt'] - df['Amt2']

date_menu = df.groupby('text_date').count()
default = date_menu.index.tolist()
default.append('QTD')
default.append('YTD')

d = widgets.Dropdown(
    options=default,
    value=default[2],
    description='Date: ',
    disabled=False,
)

dep_menu = df.groupby('Dep_Name').count()
default_d = dep_menu.index.tolist()

dep = widgets.Dropdown(
    options=default_d,
    value=default_d[0],
    description='Account: ',
    disabled=False,
)

item_menu = df[(df['text_date'] == d.value) & (df['Dep_Name'] == dep.value)]['Item_name'].unique()
default_i = item_menu.tolist()[0]

i = widgets.Dropdown(
    options=item_menu.tolist(),
    value=default_i,
    description='Item:',
    disabled=False,
)

button = widgets.Button(description="Click to Export")


def on_change(change):
    clear_output()
    display(d, dep, i, button)
    
    if (d.value == 'QTD') or (d.value == 'YTD'):
        date = df[(df['Dep_Name'] == dep.value) & (df['text_date'] >= date_dict[d.value])][['text_date','Item_name','Item', 'Total']]
        show = date.groupby(['text_date', 'Item_name']).sum()
        show_f = show.unstack(level=0)
        show_f = show_f.fillna(0)
        show_f.columns = show_f.columns.droplevel(0)
        show_f.columns.name = None
        show_f['Total'] = show_f.apply(lambda x: sum(x[show_f.columns]), axis=1)
        show_f.loc['Total',:] = show_f.apply(np.sum, axis=0)
        show_f = show_f.applymap(lambda x: '${0:,.2f}'.format(x) if x>0 else '$({0:,.2f})'.format(abs(x)))
        output = df[(df['text_date'] >= beg_date_Q) & (df['Dep_Name'] == dep.value) & (df['Item_name'] == i.value)][['Item', 'Total']]
        output = output.reset_index()
        output = output.set_index('Item')
        output = output.drop('index', axis=1)
        output = output.applymap(lambda x: '${0:,.2f}'.format(x) if x>0 else '$({0:,.2f})'.format(abs(x)))
        print(show_f)
        print("\n=================================================================================")
        print(output)
        print("\n=================================================================================")
    else:
        date = df[(df['Dep_Name'] == dep.value) & (df['text_date'] == d.value)][['text_date','Item_name','Item', 'Total']]
        show = date.groupby(['text_date', 'Item_name']).sum()
        show_f = show.unstack(level=0)
        show_f = show_f.fillna(0)
        show_f.columns = show_f.columns.droplevel(0)
        show_f.columns.name = None
        show_f.loc['Total',:] = show_f.apply(np.sum, axis=0)
        show_f = show_f.applymap(lambda x: '${0:,.2f}'.format(x) if x>0 else '$({0:,.2f})'.format(abs(x)))
        output = df[(df['text_date'] == d.value) & (df['Dep_Name'] == dep.value) & (df['Item_name'] == i.value)][['Item', 'Total']]
        output = output.reset_index()
        output = output.set_index('Item')
        output = output.drop('index', axis=1)
        output = output.applymap(lambda x: '${0:,.2f}'.format(x) if x>0 else '$({0:,.2f})'.format(abs(x)))
        print(show_f)
        print("\n=================================================================================")
        print(output)
        print("\n=================================================================================")
    return (show_f, output)
        
def on_button_clicked(b):
    root = tk.Tk()
    root.withdraw()
    try:
        dirname = filedialog.askdirectory(parent=root,initialdir="/",title='Please select a directory')
        name = dep.value+"_"+d.value[:2]+"_Summary.xlsx"
        writer = pd.ExcelWriter(dirname+ "\\" + name)
        show_f = on_change(d)[0]
        output = on_change(d)[1]
        show_f.to_excel(writer,'Sheet1')
        output.to_excel(writer,'Sheet2')
        writer.save()
        print("File will be saved to current directory with the name: " + name)
    except:
        print("No directory selected")
    

display(d, dep, i, button)
d.observe(on_change)
dep.observe(on_change)
i.observe(on_change)
button.on_click(on_button_clicked)


Dropdown(description='Date: ', index=2, options=('09/18', '10/18', '11/18', 'QTD', 'YTD'), value='11/18')

Dropdown(description='Account: ', options=('Acct', 'Acq', 'Admin', 'Asset Mgmt', 'BDREX', 'Corp Mgmt', 'Finance', 'HR', 'IR', 'IS', 'IT', 'Legal', 'Mkt', 'No Dept', 'Port Mgmt', 'Tax'), value='Acct')

Dropdown(description='Item:', options=('PAYROLL', 'PAYROLL TAXES', 'HEALTH & DENTAL', 'RETIREMENT', 'EDUCATION', 'Payroll - BRE dividends', 'TRAVEL', 'MEALS AND ENTERTAINMENT', 'SUBSCRIPTIONS', 'CONSULTING- ACCOUNTING', 'CORPORATE FUNCTIONS', 'DEPARTMENT OUTINGS', 'BANK FEES', 'OFFICE EQPT EXPENSE', 'CELL PHONES'), value='PAYROLL')

Button(description='Click to Export', style=ButtonStyle())